In [2]:
import yaml
import asyncio
import nest_asyncio
from Pricing4API.ancillary.yaml_serialization import PricingYamlHandler
from Pricing4API.ancillary.limit import Limit
from Pricing4API.ancillary.time_unit import TimeDuration, TimeUnit
from Pricing4API.main.plan import Plan
from Pricing4API.main.subscription import Subscription
PricingYamlHandler.load()

# Explorando la API de DBLP: Introducción y objetivos

El propósito de este notebook es explorar los límites asociados a cualquier API, como ejemplo usaremos la API del DBLP. Una vez entendido el alumno podrá experimentar esto mismo con cualquier otra API, siempre que conozca (o asuma) los límites asociados a ella.

## ¿Qué es DBLP?
DBLP es una base de datos bibliográfica en línea que proporciona información sobre publicaciones científicas, principalmente en el ámbito de la informática. Esta API permite acceder a datos estructurados sobre artículos, autores y conferencias, facilitando la búsqueda y el análisis de publicaciones académicas.

[Sitio web oficial de DBLP](https://dblp.org/)

---

- ¿Qué problemas puede enfrentar una API si no establece límites de consumo?
- ¿Cómo podría afectar esto a los usuarios finales o a los desarrolladores?

---



## ¿Cómo consumiríamos la API sin una especificación clara?
La API de DBLP no tiene una especificación formal sobre sus límites de consumo (*rate limits*) o cuotas. Sin embargo, en el siguiente [FAQ de DBLP](https://dblp.org/faq/Am+I+allowed+to+crawl+the+dblp+website.html), se indica que las solicitudes deben espaciarse cada 2 segundos. Seguimos sin un límite claro de solicitudes por minuto, pero tras realizar pruebas, se ha observado que los límite asumibles son de **7 solicitudes por minuto** y **14 solicitudes por minuto**.

Vamos a probar un consumo regulado durante **2 minutos y medio**, espaciando las solicitudes cada 3 segundos. Si los límites son correctos, no deberíamos obtener errores 429.

# 📘 Explicación del YAML para DBLP Pricing

Este archivo YAML define las reglas de pricing y limitaciones para consumir cualquier API. A continuación, se detallan los principales bloques y su significado.

---

## 🌟 **Estructura General**

El YAML está compuesto por los siguientes elementos clave:

1. **`!Pricing`**: Declara el tipo de configuración, en este caso relacionada con precios.
2. **`DBLP Pricing`**: Es el nombre principal de la configuración para la API de DBLP.
3. **`metrics`**: Define qué se está midiendo; aquí, el número de solicitudes (`requests`).
4. **`plans`**: Contiene los planes de suscripción, con límites y costos asociados.

---

## 🧩 **Detalles de los Bloques**

### **1. Bloque `metrics`**
```yaml
metrics:
  name: requests
```

Representa el nombre de la metrica que se esta midiendo. En este caso, se miden las solicitudes.

### **2. Bloque `plans`**

Dentro de este bloque, describiremos los diferentes planes de suscripción disponibles.

En este caso hemos definido un único plan llamado `Free DBLP`. Se podrían definir más de uno, pero siempre dentro de cada plan debemos encontrar los siguientes bloques:

---

#### **Bloque `cost` y `billing_cycle**

```yaml
  cost: 0.0
  billing_cycle:
    value: 1
    unit: month
```


Representa el costo de la suscripción. En este caso, el plan `Free DBLP` no tiene costos de suscripción.
El ciclo de facturación es de **1 mes**.

#### **Bloque `unitary_rate`**

```yaml
  unitary_rate: 
    /*:
      all:
        requests:
          period:
            value: 3
            unit: second
```

Representa el lapso de tiempo entre dos solicitudes consecutivas. En este caso, la frecuencia de las solicitudes es de **3 segundos**. 

#### **Bloque `quotas`**

```yaml
  quotas:
    /*:
      all:
        requests:
          - max: 7
            period:
              value: 1
              unit: minute
          - max: 14
            period:
              value: 5
              unit: minute
```

Representa los límites del consumo de la API del DBLP, se podrían añadir más cuotas siguiendo el mismo formato.

##### **Uso del 'globbing'**

El uso de **/*** indica que aplica a todos los endpoints, y el uso de **all** que aplica a todo metodo HTTP. En este caso se deja así por simplificar la construcción.











In [3]:
dblp_yaml = """
!Pricing
DBLP Pricing:
  metrics:
    name: requests
  plans:
    Free DBLP:
      cost: 0.0
      billing_cycle:
        value: 1
        unit: month
      unitary_rate: 
        /*:
          all:
            requests:
              period:
                value: 3
                unit: second
      quotas:
        /*:
          all:
            requests:
              - max: 7
                period:
                  value: 1
                  unit: minute
              - max: 14
                period:
                  value: 5
                  unit: minute
"""


Ahora carguemos el pricing y extraigamos el plan con el que queremos realizar todas las pruebas.

In [ ]:
pricing_DBLP = yaml.safe_load(dblp_yaml)

planes = pricing_DBLP.plans

for index, plan in enumerate(planes):
    print(plan.name," index: " , index)

El siguiente código simula el consumo de la API de DBLP, al crear un objeto Subscription por defecto regulamos el consumo, y todas las llamadas respetarán los límites establecidos. Probemos a simular 2 minutos y medio de consumo.


In [ ]:
nest_asyncio.apply()

free_dblp = planes[0] #type: Plan
dblp_subscription = Subscription(free_dblp, 'https://dblp.org/search/publ/api')
time_simulation = TimeDuration(2.5, TimeUnit.MINUTE)

asyncio.run(dblp_subscription.api_usage_simulator_async(time_simulation))

¿Obtenemos el resultado esperado? Para entenderlo mejor vamos a introducir el concepto de curva de capacidad.

---

- ¿Qué es una curva de capacidad?
- ¿Cómo se relaciona con los límites de consumo de una API?

---

Es una representación del número máximo de solicitudes que podemos realizar a lo largo del tiempo, considerando las cuotas del plan y los intervalos de tiempo establecidos. Nos muestra cuántas solicitudes podríamos haber consumido en un momento dado si respetamos estrictamente los límites.


In [ ]:
free_dblp.show_available_capacity_curve(TimeDuration(2.5, TimeUnit.MINUTE))

**¿Hemos respetado los límites de consumo de la API de DBLP?**

En la simulación previa no hemos obtenido errores 429, aún así para realmente garantizar que no hemos excedido los límites, la curva de demanada (el consumo real de las solicitudes) siempre debe mantenerse por **debajo o igual** a la curva de capacidad ideal. De lo contrario, podríamos exceder los límites de consumo y recibir errores 429.

**En color azul vemos como se muestra la curva de la demanda, y en verde la curva de la capacidad ideal.**



In [ ]:
dblp_subscription.demand_curve_vs_ideal_capacity()

El código anterior tiene en cuenta todo el tiempo que estamos suscritos, por tanto la curva sigue creciendo conforme pasa el tiempo. Mostrando la demanda actual de las solicitudes.

Para observar en el siguiente ejemplo podemos observar una curva limitada a la duración que deseemos, sin que crezca con el tiempo. Prueba a cambiar la simulación a 1 minuto y este código siempre devolvería la misma curva.

In [ ]:
dblp_subscription.demand_curve_vs_ideal_capacity(shortened=True, time_simulation=TimeDuration(2.5, TimeUnit.MINUTE))

---

## ¿Cuántas solicitudes puedes hacer?

Basándonos en lo que sabemos sobre la API, intenta responder a las siguientes preguntas. Usa los valores de **rate** y **cuota** para responder:

1. ¿Cuántas solicitudes se pueden hacer en:
   - 10 segundos?
   - 1 minuto?
   - 1 minuto y 1 segundo?

---

Para responder con precisión a estas preguntas, necesitamos una función que calcule el número máximo de solicitudes permitidas en un período \( t \), respetando tanto el **rate** como la **cuota** del plan.


### Ejercicio: Implementar capacity(t)

La función `capacity(t)` ya está implementada en la clase `Plan`. Vamos a probarla en distintos escenarios:

In [ ]:
time_simulation_9s = TimeDuration(9, TimeUnit.SECOND)
time_simulation_59s = TimeDuration(59, TimeUnit.SECOND)
time_simulation_1min = TimeDuration(1, TimeUnit.MINUTE)
time_simulation_1min_1s = TimeDuration(1, TimeUnit.MINUTE) + TimeDuration(1, TimeUnit.SECOND)

capacity_9s = free_dblp.available_capacity(time_simulation_9s, len(free_dblp.limits)-1)
capacity_59s = free_dblp.available_capacity(time_simulation_59s, len(free_dblp.limits)-1)
capacity_1min = free_dblp.available_capacity(time_simulation_1min, len(free_dblp.limits)-1)
capacity_1min_1s = free_dblp.available_capacity(time_simulation_1min_1s, len(free_dblp.limits)-1)

print(f"Capacidad para 9 segundos: {capacity_9s}")
print(f"Capacidad para 59 segundos: {capacity_59s}")
print(f"Capacidad para 1 minuto: {capacity_1min}")
print(f"Capacidad para 1 minuto y 1 segundo: {capacity_1min_1s}")



## ¡Prueba con los valores que tú quieras!

In [ ]:
time_simulation =

## ¿Cuánto tiempo necesitas para hacer X solicitudes?

Usando los límites de la API (rate y cuota), intenta calcular cuánto tiempo tomaría realizar las siguientes solicitudes. Piensa si el rate o la cuota influirán más en el resultado:

1. ¿Cuánto tiempo necesitas para realizar:
   - 7 solicitudes?
   - 8 solicitudes?
   - 14 solicitudes?

---

Para responder estas preguntas, necesitamos una función, que calcule el tiempo mínimo necesario para realizar un número dado de solicitudes, respetando tanto el **rate** como la **cuota** del plan.

### Ejercicio: min_time(capacity_goal)

La función `min_time(capacity_goal)` ya está implementada en la clase `Plan`. Vamos a probarla en distintos escenarios:

In [ ]:
# Tiempo necesario para realizar 7, 8 y 14 solicitudes
time_7_requests = free_dblp.min_time(7, display=True)
time_8_requests = free_dblp.min_time(8, display=True)
time_14_requests = free_dblp.min_time(14, display=True)

print(f"Tiempo para 7 solicitudes: {time_7_requests}")
print(f"Tiempo para 8 solicitudes: {time_8_requests}")
print(f"Tiempo para 14 solicitudes: {time_14_requests}")

## ¡Prueba con los valores que tú quieras!

In [ ]:
time_x_requests = free_dblp.min_time(1000)

print(f"Tiempo para 100 solicitudes: {time_x_requests}")


# EJERCICIO: Prueba ahora con la siguiente API

Constrúyete tu propio Pricing con un Plan inventado.

Usa el siguiente endpoint `https://www.el-tiempo.net/api/json/v2/provincias`

Sigue los pasos anteriores e intenta ver la respuesta que obtienes. Prueba con los límites, verás la curva de la demanda y de la capacidad ideal, y te encontrarás con los problemas de no especificar los límites 😉.

Puedes probar a eliminar el rate unitario, jugar con las cuotas...

In [ ]:
eltiempo_yaml = """
!Pricing
ElTiempo Pricing:
  metrics:
    name: requests
  plans:
    Free Tiempo:
      cost: 0.0
      billing_cycle:
        value: 1
        unit: month
      unitary_rate: 
        /*:
          all:
            requests:
              period:
                value: 
                unit:
      quotas:
        /*:
          all:
            requests:

        
"""

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bfc07943-bcc8-48e2-8be3-26469867b14f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>